# Ranking buildings with flexible potential for all houses (general knowlage vs IRISE data)

1) Irise data ranking houses acording its flexibility potentials

Ranking for 1 year (its possible to filter it for hours or days or months and so on) and for all houses. IRISE data (it is posible to filter them with houses also it is posible to drop houses which do not have reasonable data) (complex indicators)

In [4]:
import pandas as pd
import plotly.graph_objects as go

# Load the Excel file for total energy consumption
excel_file_5 = pd.ExcelFile(r'D:\Grenoble_university\6_month_internship\working space\data\combined_1_year_data_with_flexibility.xlsx')

# Define the range of sheet numbers for total energy consumption
start_sheet_number = 2000900
end_sheet_number = 2000997

# Initialize total consumption dictionary
total_consumption = {}

# Define houses to be excluded
excluded_houses = {'2000902', '2000920', '2000919', '2000924', '2000933', '2000937', '2000977'}

# Iterate through the sheet numbers for total energy consumption
for sheet_number in range(start_sheet_number, end_sheet_number + 1):
    sheet_name = str(sheet_number)
    if sheet_name in excel_file_5.sheet_names and sheet_name not in excluded_houses:
        # Read the sheet into a DataFrame
        df = excel_file_5.parse(sheet_name)
        
        # Check if 'Site consumption ()' is in the columns of the DataFrame
        if 'Site consumption ()' in df.columns:
            # Convert 'Site consumption ()' column to numeric (ignore errors)
            df['Site consumption ()'] = pd.to_numeric(df['Site consumption ()'], errors='coerce')
            # Calculate the sum for 'Site consumption ()' column
            sum_for_site_consumption = df['Site consumption ()'].sum()
            total_consumption[sheet_name] = sum_for_site_consumption
        else:
            # Sum up all numeric columns
            sum_for_devices = df.iloc[1:].drop(columns=['Weekday', 'date']).sum().sum()
            total_consumption[sheet_name] = sum_for_devices
    else:
        print(f"Sheet {sheet_name} not found in the Excel file or excluded.")

# Convert total consumption dictionary to DataFrame
total_consumption_df = pd.DataFrame.from_dict(total_consumption, orient='index', columns=['Total Consumption'])

# Read the Excel file into a dictionary of DataFrames for energy flexibility during peak hours
combined_data_dict_5 = pd.read_excel(r'D:\Grenoble_university\6_month_internship\working space\data\combined_1_year_data_with_flexibility.xlsx', sheet_name=None)

# Define the time range for calculation (8 am - 10 am and 6 pm - 10 pm)
start_time_morning = pd.Timestamp('00:00:00').time()
end_time_morning = pd.Timestamp('12:00:00').time()
start_time_evening = pd.Timestamp('12:00:01').time()
end_time_evening = pd.Timestamp('23:59:59').time()

# Initialize lists to store total morning and evening flexible consumption
total_flex_consumption = []

# Define a function to filter data by weekdays
def filter_by_weekdays(df, weekdays):
    # Convert 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'])
    # Create a dictionary to map weekday names to day numbers
    weekday_map = {
        'Monday': 0,
        'Tuesday': 1,
        'Wednesday': 2,
        'Thursday': 3,
        'Friday': 4,
        'Saturday': 5,
        'Sunday': 6
    }
    # Filter the DataFrame to include only the specified weekdays
    day_numbers = [weekday_map[day] for day in weekdays]
    return df[df['date'].dt.dayofweek.isin(day_numbers)]

# Specify the weekdays you are interested in
desired_weekdays = ['Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday', 'Monday']

# Iterate over each sheet and perform the filtering and calculation for energy flexibility during peak hours
for sheet_name, df in combined_data_dict_5.items():
    if sheet_name not in excluded_houses:
        # Filter rows to include only flexible devices (second row)
        flexibility_row = df.iloc[0] == 'flexible'
        flexible_devices_df = df.loc[:, flexibility_row]

        # Convert 'date' column to datetime format
        df['date'] = pd.to_datetime(df['date'])

        # Filter flexible devices data by desired weekdays
        filtered_df = filter_by_weekdays(df, desired_weekdays)
        flexible_devices_df = flexible_devices_df.loc[filtered_df.index]

        # Filter rows based on time range and calculate sum for morning and evening
        morning_sum = flexible_devices_df[
            (filtered_df['date'].dt.time >= start_time_morning) & 
            (filtered_df['date'].dt.time <= end_time_morning)
        ].sum().sum()

        evening_sum = flexible_devices_df[
            (filtered_df['date'].dt.time >= start_time_evening) & 
            (filtered_df['date'].dt.time <= end_time_evening)
        ].sum().sum()

        # Append total morning and evening sums to list
        total_flex_consumption.append(morning_sum + evening_sum)
    else:
        print(f"Sheet {sheet_name} excluded.")

# Create a DataFrame to store the flexible consumption data
flex_consumption_df = pd.DataFrame({
    'House ID': [k for k in combined_data_dict_5.keys() if k not in excluded_houses],
    'Total Flexible Consumption': total_flex_consumption
})

# Convert values to kWh
total_consumption_df['Total Consumption'] /= 1_000
flex_consumption_df['Total Flexible Consumption'] /= 1_000

# Merge the total consumption and flexible consumption DataFrames
merged_df = total_consumption_df.merge(flex_consumption_df, left_index=True, right_on='House ID')

# Define flexibility thresholds
high_threshold = merged_df['Total Flexible Consumption'].quantile(0.66)
low_threshold = merged_df['Total Flexible Consumption'].quantile(0.33)

# Categorize flexibility potential
def categorize_flexibility(value):
    if value >= high_threshold:
        return 'High'
    elif value >= low_threshold:
        return 'Average'
    else:
        return 'Low'

merged_df['Flexibility Category'] = merged_df['Total Flexible Consumption'].apply(categorize_flexibility)

# Define colors based on flexibility categories
color_map = {
    'High': 'green',
    'Average': 'yellow',
    'Low': 'red'
}

merged_df['Color'] = merged_df['Flexibility Category'].map(color_map)

# Sort DataFrame by Total Flexible Consumption
merged_df = merged_df.sort_values(by='Total Flexible Consumption')

# Create separate DataFrames for each flexibility category
high_df = merged_df[merged_df['Flexibility Category'] == 'High']
average_df = merged_df[merged_df['Flexibility Category'] == 'Average']
low_df = merged_df[merged_df['Flexibility Category'] == 'Low']



Sheet 2000902 not found in the Excel file or excluded.
Sheet 2000919 not found in the Excel file or excluded.
Sheet 2000920 not found in the Excel file or excluded.
Sheet 2000924 not found in the Excel file or excluded.
Sheet 2000933 not found in the Excel file or excluded.
Sheet 2000937 not found in the Excel file or excluded.
Sheet 2000977 not found in the Excel file or excluded.
Sheet 2000902 excluded.
Sheet 2000919 excluded.
Sheet 2000920 excluded.
Sheet 2000924 excluded.
Sheet 2000933 excluded.
Sheet 2000937 excluded.
Sheet 2000977 excluded.


In [7]:
# Create a figure
fig = go.Figure()

# Add bars for total consumption
fig.add_trace(go.Bar(
    x=merged_df['House ID'],
    y=merged_df['Total Consumption'],
    name='Total Consumption',
    marker_color='blue'
))

# Add bars for flexible consumption with colors based on flexibility category
fig.add_trace(go.Bar(
    x=low_df['House ID'],
    y=low_df['Total Flexible Consumption'],
    name='Low Flexibility Consumption',
    marker_color='red',
    text=low_df['Flexibility Category'],
    opacity=0.5
))

fig.add_trace(go.Bar(
    x=average_df['House ID'],
    y=average_df['Total Flexible Consumption'],
    name='Average Flexibility Consumption',
    marker_color='yellow',
    text=average_df['Flexibility Category'],
    opacity=0.5
))

fig.add_trace(go.Bar(
    x=high_df['House ID'],
    y=high_df['Total Flexible Consumption'],
    name='High Flexibility Consumption',
    marker_color='green',
    text=high_df['Flexibility Category'],
    opacity=0.5
))

# Update layout with appropriate y-axis range and formatting
fig.update_layout(
    title='Total Energy Consumption vs Flexible Consumption',
    xaxis_title='House ID',
    yaxis_title='Energy Consumption (kWh)',
    yaxis=dict(
        tickformat=',.0f',  # Use comma as a thousand separator
        range=[0, 18000]    # Set y-axis range to a maximum of 18,000
    ),
    barmode='overlay',
    xaxis_tickangle=-45,
    width=2000,  # Increase width to fit more house IDs
    height=800,  # Increase height for better readability
    margin=dict(
        l=100,   # left margin
        r=100,   # right margin
        b=200,  # bottom margin to accommodate x-axis labels
        t=100   # top margin for title
    ),
    legend_traceorder='reversed'  # Reverse the order of items in the legend
)

# Show the figure
fig.show()

# Simple indicators
Ranking for 1 year indicators acording its flexibility conditions. (for simple indicators)

experiment n 1 (I am trying to have sum of site consumption from iris data)

In [26]:
import pandas as pd
import plotly.graph_objs as go

# Load the Excel file with the correct sheet name
excel_file = pd.ExcelFile(r'D:\Grenoble_university\6_month_internship\working space\data\combined_1_year_data_with_flexibility.xlsx')

# Load the appliances data
appliances_file_path = r'D:\Grenoble_university\6_month_internship\working space\data\column_titles_with add info.xlsx'
appliances_df = pd.read_excel(appliances_file_path)

# Define houses to be excluded
excluded_houses = {'2000902', '2000920', '2000919', '2000924', '2000933', '2000937', '2000977'}

# Initialize dictionaries to store total and flexible energy consumption
total_consumption = {}
flexible_consumption = {}
total_site_consumption = {}
Total_light_consumption = {}

# Iterate over each sheet from 2000900 to 2000997
for sheet_number in range(2000900, 2000997):
    sheet_name = str(sheet_number)
    if sheet_name in excel_file.sheet_names and sheet_name not in excluded_houses:
        print(f"Processing sheet {sheet_name}")
        # Read the sheet
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        
        # Initialize dictionaries for this sheet
        total_consumption[sheet_name] = 0
        flexible_consumption[sheet_name] = 0
        total_site_consumption[sheet_name] = 0
        Total_light_consumption[sheet_name] = 0
        
        # Iterate over columns and check if appliance names match
        for column in df.columns:
            for index, appliance in appliances_df.iterrows():
                # Extract substring before "(" symbol for comparison
                appliance_name = appliance['Appliance']
                if appliance_name in column:
                    print(f"Found matching appliance: {appliance_name}")
                    # Convert column data to numeric type
                    df[column] = pd.to_numeric(df[column], errors='coerce')
                    # Calculate annual energy consumption based on typical power
                    Total_annual_energy = df[column].sum()  # Sum the column to get total annual energy consumption
                    annual_energy = appliance['typical annual energy [kwh/yr]']
                    # Categorize as flexible or not flexible based on flexibility in appliances data
                    flexibility = appliance['Flexibility']
                    if flexibility == 'flexible':
                        print(f"{appliance['Appliance']} is flexible with annual energy consumption: {annual_energy} kWh")
                        flexible_consumption[sheet_name] += annual_energy
                        #total_consumption[sheet_name] += annual_energy  # Add flexible consumption to total
                    elif flexibility == 'Uncategorized' and appliance_name == 'Site consumption':
                        print(f"{appliance['Appliance']} is total site consumption (average values): {Total_annual_energy} kWh")
                        total_site_consumption[sheet_name] += Total_annual_energy/1000
                    elif flexibility == 'Uncategorized' and appliance_name == 'Total site light consumption':
                        print(f"{appliance['Appliance']} is total light consumption (average values): {annual_energy} kWh")
                        #Total_light_consumption[sheet_name] += annual_energy
                    else:
                        print(f"{appliance['Appliance']} is not flexible with annual energy consumption: {annual_energy} kWh")
                        total_consumption[sheet_name] += annual_energy  # Add non-flexible consumption to total

        # Add flexible consumption to total consumption
        #total_consumption[sheet_name] += flexible_consumption[sheet_name]

# Convert dictionaries to DataFrame
data_df = pd.DataFrame({
    'House ID': list(total_site_consumption.keys()),
    'Total Consumption': list(total_site_consumption.values()),
    'Flexible Consumption': list(flexible_consumption.values())
})

# Define flexibility thresholds
high_threshold = data_df['Flexible Consumption'].quantile(0.66)
low_threshold = data_df['Flexible Consumption'].quantile(0.33)

# Categorize flexibility potential
def categorize_flexibility(value):
    if value >= high_threshold:
        return 'High'
    elif value >= low_threshold:
        return 'Average'
    else:
        return 'Low'

data_df['Flexibility Category'] = data_df['Flexible Consumption'].apply(categorize_flexibility)

# Define colors based on flexibility categories
color_map = {
    'High': 'green',
    'Average': 'yellow',
    'Low': 'red'
}

data_df['Color'] = data_df['Flexibility Category'].map(color_map)

# Sort DataFrame by Flexible Consumption
data_df = data_df.sort_values(by='Flexible Consumption')

# Create a figure
fig = go.Figure()

# Add bars for total consumption
fig.add_trace(go.Bar(
    x=data_df['House ID'],
    y=data_df['Total Consumption'],
    name='Total Consumption',
    marker_color='blue'
))

# Add bars for flexible consumption with colors based on flexibility category
fig.add_trace(go.Bar(
    x=data_df['House ID'],
    y=data_df['Flexible Consumption'],
    name='Flexible Consumption',
    marker=dict(color=data_df['Color']),
    opacity=0.5
))

# Update layout
fig.update_layout(
    title='Total Energy Consumption vs. Flexible Consumption',
    xaxis_title='House ID',
    yaxis_title='Energy Consumption (kWh)',
    yaxis=dict(
        tickformat=',.0f',
        range=[0, 25000]
    ),
    barmode='overlay',
    xaxis_tickangle=-45,
    width=2000,
    height=800,
    margin=dict(
        l=100,
        r=100,
        b=200,
        t=100
    )
)

# Show the figure
fig.show()


Processing sheet 2000900
Found matching appliance: Fridge
Fridge is not flexible with annual energy consumption: 346 kWh
Found matching appliance: Halogen lamp
Halogen lamp is flexible with annual energy consumption: 134 kWh
Found matching appliance: Power supply for wood boiler
Power supply for wood boiler is flexible with annual energy consumption: 120 kWh
Found matching appliance: Site consumption
Site consumption is total site consumption (average values): 5433955.0 kWh
Found matching appliance: Total site light consumption
Total site light consumption is total light consumption (average values): 443 kWh
Found matching appliance: Vertical freezer
Vertical freezer is not flexible with annual energy consumption: 296 kWh
Found matching appliance: Washing machine
Washing machine is flexible with annual energy consumption: 100 kWh
Found matching appliance: Water heater
Water heater is flexible with annual energy consumption: 1676 kWh
Processing sheet 2000901
Found matching appliance: Ch

Experiment 2 (total consumption is just sum of all electrical devices does not matter if it is flexible or not)

In [43]:
import pandas as pd
import plotly.graph_objs as go

# Load the Excel file with the correct sheet name
excel_file = pd.ExcelFile(r'D:\Grenoble_university\6_month_internship\working space\data\combined_1_year_data_with_flexibility.xlsx')

# Load the appliances data
appliances_file_path = r'D:\Grenoble_university\6_month_internship\working space\data\column_titles_with add info.xlsx'
appliances_df = pd.read_excel(appliances_file_path)

# Define houses to be excluded
excluded_houses = {'2000902', '2000920', '2000919', '2000924','2000933', '2000937', '2000977'} #('2000902', '2000920', '2000919', '2000924' site consumption is less then sum of flexible devices), ('2000933', '2000937', '2000977' without site consumption)

# Initialize dictionaries to store total and flexible energy consumption
total_consumption = {}
flexible_consumption = {}
total_site_consumption = {}
Total_light_consumption = {}

# Iterate over each sheet from 2000900 to 2000997
for sheet_number in range(2000900, 2000998):
    sheet_name = str(sheet_number)
    if sheet_name in excel_file.sheet_names and sheet_name not in excluded_houses:
        print(f"Processing sheet {sheet_name}")
        # Read the sheet
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        
        # Initialize dictionaries for this sheet
        total_consumption[sheet_name] = 0
        flexible_consumption[sheet_name] = 0
        total_site_consumption[sheet_name] = 0
        Total_light_consumption[sheet_name] =0
        
        # Iterate over columns and check if appliance names match
        for column in df.columns:
            for index, appliance in appliances_df.iterrows():
                # Extract substring before "(" symbol for comparison
                appliance_name = appliance['Appliance'].split("(")[0].strip()
                if appliance_name in column:
                    print(f"Found matching appliance: {appliance_name}")
                    # Convert column data to numeric type
                    df[column] = pd.to_numeric(df[column], errors='coerce')
                    # Calculate annual energy consumption based on typical power
                    # typical_power = appliance['typical power']
                    annual_energy = appliance['typical annual energy [kwh/yr]']                       # typical_power * df[column].sum() / (60 * 24 * 365)  # Assuming data is in 1-minute intervals
                    # Categorize as flexible or not flexible based on flexibility in appliances data
                    flexibility = appliance['Flexibility']
                    if flexibility == 'flexible':
                        print(f"{appliance['Appliance']} is flexible with annual energy consumption: {annual_energy} kWh")
                        flexible_consumption[sheet_name] += annual_energy
                    elif flexibility == 'not_flexible':
                        print(f"{appliance['Appliance']} is not flexible with annual energy consumption: {annual_energy} kWh")
                        total_consumption[sheet_name] += annual_energy
        # Add flexible consumption to total consumption
        total_consumption[sheet_name] += flexible_consumption[sheet_name]

# Convert dictionaries to DataFrame
data_df = pd.DataFrame({
    'House ID': list(total_consumption.keys()),
    'Total Consumption': list(total_consumption.values()),
    'Flexible Consumption': list(flexible_consumption.values())
})

# Define flexibility thresholds
high_threshold = data_df['Flexible Consumption'].quantile(0.66)
low_threshold = data_df['Flexible Consumption'].quantile(0.33)

# Categorize flexibility potential
def categorize_flexibility(value):
    if value >= high_threshold:
        return 'High'
    elif value >= low_threshold:
        return 'Average'
    else:
        return 'Low'

data_df['Flexibility Category'] = data_df['Flexible Consumption'].apply(categorize_flexibility)

# Define colors based on flexibility categories
color_map = {
    'High': 'green',
    'Average': 'yellow',
    'Low': 'red'
}

data_df['Color'] = data_df['Flexibility Category'].map(color_map)

# Sort DataFrame by Flexible Consumption
data_df = data_df.sort_values(by='Flexible Consumption')

# Create a figure
fig = go.Figure()

# Add bars for total consumption
fig.add_trace(go.Bar(
    x=data_df['House ID'],
    y=data_df['Total Consumption'],
    name='Total Consumption',
    marker_color='blue'
))

# Add bars for flexible consumption with colors based on flexibility category
fig.add_trace(go.Bar(
    x=data_df['House ID'],
    y=data_df['Flexible Consumption'],
    name='Flexible Consumption',
    marker=dict(color=data_df['Color']),
    opacity=0.5
))

# Update layout
fig.update_layout(
    title='Total Energy Consumption vs. Flexible Consumption',
    xaxis_title='House ID',
    yaxis_title='Energy Consumption (kWh)',
    yaxis=dict(
        tickformat=',.0f',
        range=[0, 25000]
    ),
    barmode='overlay',
    xaxis_tickangle=-45,
    width=2000,
    height=800,
    margin=dict(
        l=100,
        r=100,
        b=200,
        t=100
    )
)

# Show the figure
fig.show()


Processing sheet 2000900
Found matching appliance: Fridge
Fridge is not flexible with annual energy consumption: 346 kWh
Found matching appliance: Halogen lamp
Halogen lamp is flexible with annual energy consumption: 134 kWh
Found matching appliance: Power supply for wood boiler
Power supply for wood boiler is flexible with annual energy consumption: 120 kWh
Found matching appliance: Site consumption
Found matching appliance: Total site light consumption
Found matching appliance: Vertical freezer
Vertical freezer is not flexible with annual energy consumption: 296 kWh
Found matching appliance: Washing machine
Washing machine is flexible with annual energy consumption: 100 kWh
Found matching appliance: Water heater
Water heater is flexible with annual energy consumption: 1676 kWh
Processing sheet 2000901
Found matching appliance: Chest freezer
Chest freezer is not flexible with annual energy consumption: 296 kWh
Found matching appliance: Clothes drier
Clothes drier is flexible with annu

# Comparison matching: Simple and complex flexible houses 

In [52]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assuming flexibility_code1 and flexibility_code2 are already computed from Code 1 and Code 2 respectively

# Extract relevant columns from data_df (Code 1)
flexibility_simple_1_df = data_df[['House ID', 'Flexibility Category']].rename(columns={'Flexibility Category': 'Flexibility_simple_1'})

# Extract relevant columns from merged_df (Code 2)
flexibility_complex_2_df = merged_df[['House ID', 'Flexibility Category']].rename(columns={'Flexibility Category': 'Flexibility_complex_2'})

# Merge flexibility categories from both codes based on House ID
flexibility_comparison = pd.merge(flexibility_simple_1_df, flexibility_complex_2_df, on='House ID', how='outer')

# Fill NaN values with 'Unknown' for houses not present in both codes
flexibility_comparison = flexibility_comparison.fillna('Unknown')

# Identify matching and non-matching flexibility categories
flexibility_comparison['Match'] = flexibility_comparison['Flexibility_simple_1'] == flexibility_comparison['Flexibility_complex_2']

# Calculate total number of houses and number of matching houses
total_houses = len(flexibility_comparison)
matching_houses = flexibility_comparison['Match'].sum()

# Calculate percentage of matching houses
percentage_matching = (matching_houses / total_houses) * 100
percentage_non_matching = 100 - percentage_matching

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Output the results
print(f"Total houses considered: {total_houses}")
print(f"Number of matching houses: {matching_houses}")
print(f"Percentage of matching houses: {percentage_matching:.2f}%")

# Separate houses into matched and non-matched categories
matched_houses = flexibility_comparison[flexibility_comparison['Match']]
mismatched_houses = flexibility_comparison[~flexibility_comparison['Match']]

# Display the results
print("Houses with Matching Flexibility Categories:")
print(matched_houses)

print("\nHouses with Non-Matching Flexibility Categories:")
print(mismatched_houses)

# Optionally, you can also show the details of how each house differs
if not mismatched_houses.empty:
    for index, row in mismatched_houses.iterrows():
        house_id = row['House ID']
        flexibility_code1 = row['Flexibility_simple_1']
        flexibility_code2 = row['Flexibility_complex_2']
        print(f"House {house_id}: Flexibility simple 1 - {flexibility_code1}, Flexibility complex 2 - {flexibility_code2}")

# Visualization
# Count the number of houses in each flexibility category for both codes
flexibility_simple_counts_1 = flexibility_simple_1_df['Flexibility_simple_1'].value_counts().reset_index()
flexibility_simple_counts_1.columns = ['Flexibility Category', 'Count']
flexibility_simple_counts_1['Code'] = 'Simple indicators_1'

flexibility_complex_counts_2 = flexibility_complex_2_df['Flexibility_complex_2'].value_counts().reset_index()
flexibility_complex_counts_2.columns = ['Flexibility Category', 'Count']
flexibility_complex_counts_2['Code'] = 'Complex indicators_2'

# Combine counts into a single DataFrame
flexibility_counts = pd.concat([flexibility_simple_counts_1, flexibility_complex_counts_2])

# Create bar plot to show the counts of each flexibility category for both codes
fig = px.bar(flexibility_counts, x='Flexibility Category', y='Count', color='Code', barmode='group',
             title='Comparison of Flexibility Categories from Simple indicators 1 and Complex indicators 2',
             labels={'Count': 'Number of Houses', 'Flexibility Category': 'Flexibility Category'})

fig.show()

# Create a scatter plot to visualize matching and non-matching houses
flexibility_comparison['Category Comparison'] = flexibility_comparison.apply(
    lambda row: f"{row['Flexibility_simple_1']} to {row['Flexibility_complex_2']}", axis=1
)

# Count the number of houses in each category comparison
comparison_counts = flexibility_comparison['Category Comparison'].value_counts().reset_index()
comparison_counts.columns = ['Category Comparison', 'Count']

# Calculate the percentage for each category comparison
comparison_counts['Percentage'] = (comparison_counts['Count'] / total_houses) * 100

# Create bar plot to show the counts of each category comparison
fig_comparison = px.bar(comparison_counts, x='Category Comparison', y='Count',
                        title='Comparison of Flexibility Categories between Simple indicators 1 and Complex indicators 2',
                        labels={'Count': 'Number of Houses', 'Category Comparison': 'Category Comparison'},
                        color='Count', color_continuous_scale='Viridis',
                        hover_data={'Percentage': ':.2f'})

fig_comparison.show()

# Visualization of matching and non-matching percentages
match_data = pd.DataFrame({
    'Category': ['Matching', 'Non-Matching'],
    'Count': [matching_houses, total_houses - matching_houses],
    'Percentage': [percentage_matching, percentage_non_matching]
})

fig_match = go.Figure()

# Add bars
fig_match.add_trace(go.Bar(
    x=match_data['Category'],
    y=match_data['Percentage'],
    text=match_data['Percentage'].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',
    marker_color=['#636EFA', '#EF553B']
))

# Update layout
fig_match.update_layout(
    title='Percentage of Matching and Non-Matching Houses',
    xaxis_title='Category percentage',
    yaxis_title='Percentage of Houses',
    yaxis_ticksuffix='%',
    height=500
)

fig_match.show()


Total houses considered: 91
Number of matching houses: 75
Percentage of matching houses: 82.42%
Houses with Matching Flexibility Categories:
   House ID Flexibility_simple_1 Flexibility_complex_2  Match
1   2000901              Average               Average   True
2   2000903              Average               Average   True
3   2000904                 High                  High   True
4   2000905                  Low                   Low   True
5   2000906                  Low                   Low   True
6   2000907                 High                  High   True
7   2000908              Average               Average   True
8   2000909                 High                  High   True
9   2000910                  Low                   Low   True
11  2000912                  Low                   Low   True
12  2000913                  Low                   Low   True
13  2000914                 High                  High   True
14  2000915                 High                  Hig